# 1 - MODULES AND CONSTANTS

### MODULES AND LIBRARIES

### CONSTANTS

In [69]:
REFERENCE_SEQUENCE_INPUT_PATH = 

# 2 - LOAD DATA

loads the dataFrame from a tsv file, uses the first row of said file as the column names, moreover, <br>
it drops the column organism (because it's the same for all entries since I filtered for humans), <br>
it, also, drops all the rows containing at least a missing value.


In [70]:
def readDataFrameFromTSV(path:str=DATA_PATH)->pd.DataFrame:
    df = pd.read_csv(filepath_or_buffer=path, sep='\t', header=0, index_col=0)
    return df[['Function [CC]', 'Sequence' ]].dropna(axis=0).rename(columns={'Function [CC]':'Function'})
    return pd.read_csv(filepath_or_buffer=path, sep='\t', header=0, index_col=0).drop(columns=['Organism', 'Gene Names (synonym)']).dropna(axis=0).rename(columns={'Function [CC]':'Function'})

df = readDataFrameFromTSV(DATA_PATH)

# 3 - PREPROCESSING

Performs the encoding of the labels, using sklearn label encoder, transforming categorical feature into numerical ones to be used for classification.

In [71]:
def labelEncoding(df:pd.DataFrame)->pd.DataFrame:
    df['Sequence'] = LabelEncoder().fit_transform(df['Sequence'])
    return df

df = labelEncoding(df)
df = df.head(1000)

In [72]:
pd.read_csv(filepath_or_buffer=DATA_PATH, sep='\t', header=0, index_col=0).isna().sum()

Entry Name                   0
Protein names                0
Gene Names               39199
Organism                     0
Length                       0
Function [CC]           162025
Protein existence            0
Mass                         0
Gene Names (synonym)    189142
Sequence                     0
dtype: int64

In [73]:
xTrain, xTest, yTrain, yTest = train_test_split(df[['Sequence']], df['Function'])

In [74]:
yPred = RandomForestClassifier(n_jobs=-1).fit(xTrain, yTrain).predict(xTest)

heatmap(confusion_matrix(yTest, yPred), annot=True)

accuracy_score(yTest, yPred), precision_score(yTest, yPred, average='micro'), recall_score(yTest, yPred, average='micro'), f1_score(yTest, yPred, average='micro')

(0.052, 0.052, 0.052, 0.052)

Error in callback <function flush_figures at 0x0000020895C125C0> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 

In [76]:
s1 = "CGATTCCAGGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTC"
s2 = "GGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTCGTCCAGACCCCTAGC"

def getPairs(s1,s2):
    for i in range(len(s2)-1, -1, -1):
        if not s1.endswith(s2[i:]):
            return i
        
    return -1

s2[getPairs(s1, s2):]

'GC'